In [ ]:
# TODO:
# to be rewritten as tests

In [1]:
from commutazzio.filtration import SimplicialComplex, SimplexTree
from commutazzio.filtration import CLFiltration
from commutazzio.filtration import CLFiltrationDB
from commutazzio.filtration import pointCloud2Filtration, random_vertical_removal_points_only
from commutazzio.compute import CLInvariants
from commutazzio.plot import ComplementaryTrianglesPlot as Visualizer1
import numpy as np
from icecream import ic
import pdb
import pandas as pd
from io import StringIO
from cpes import FaceCenteredCubic as FCC, HexagonalClosePacking as HCP
from commutazzio.plot import ComplementaryTrianglesPlot as Visualizer1
from commutazzio.utils import join_and_unique
EPSILON = 1e-10

In [2]:
def upper_layer_creation(crystal_type,side_length,homology_dim,radii,method):
    if crystal_type == "fcc":
        crystal=FCC(side_length)
    elif crystal_type == "hcp":
        crystal=HCP(side_length)
    else:
        raise NotImplementedError("crystal_type must be either fcc or hcp")
    st=SimplexTree()
    st.from_point_cloud(crystal.data,method=method,sc_dim_ceil=homology_dim+1,radius_max=max(radii)+EPSILON)
    upper=SimplexTree()
    for i,radius in enumerate(radii):
        x_coord=i+1
        sc=st.truncation(radius)
        for simplex in sc.simplices:
            upper.insert(simplex,x_coord)
    return upper

def lower_layer_creation(upper_layer,radii,singlet_deletion_list:list):
    lower=SimplexTree()
    for i,radius in enumerate(radii):
        x_coord=i+1
        sc=upper_layer.truncation(x_coord)
        after_deletion=sc.delete_simplices(singlet_deletion_list).simplices
        for simplex in after_deletion:
            lower.insert(simplex,x_coord)
    return lower




In [3]:
radii=np.linspace(1,2,11)
upper=upper_layer_creation("fcc",5,1,radii,"cech")
removal_list=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
lower=lower_layer_creation(upper,radii,removal_list)
aaa=CLFiltration(upper=upper,lower=lower,ladder_length=len(radii),h_params=radii,info={'singlet_removal':removal_list})

Generating close packing with atom radius 1.
CLFiltration:ordinal number filtration values detected, will not rescale.


In [4]:
inv_t=CLInvariants(aaa)

In [5]:
inv_t.cPD_computation()

Computing connected persistence diagram at dimension 1
Saving to ./filtration/20230621_152251_94e39.fltr
Loading precomputed intervals and variables...
全ての道の差分リストを構築
Saving to /Users/hina/Library/CloudStorage/OneDrive-KyotoUniversity/Documents/KU/commutative-grid/working_dir_cl50/FCC_HCP/filtration/20230621_152251_94e39_FZZ_upper.txt
Saving to /Users/hina/Library/CloudStorage/OneDrive-KyotoUniversity/Documents/KU/commutative-grid/working_dir_cl50/FCC_HCP/filtration/20230621_152251_94e39_FZZ_lower.txt
Saving to /Users/hina/Library/CloudStorage/OneDrive-KyotoUniversity/Documents/KU/commutative-grid/working_dir_cl50/FCC_HCP/filtration/20230621_152251_94e39_FZZ_0_0.txt
進捗率: 33.33％ Saving to /Users/hina/Library/CloudStorage/OneDrive-KyotoUniversity/Documents/KU/commutative-grid/working_dir_cl50/FCC_HCP/filtration/20230621_152251_94e39_FZZ_0_1.txt
進捗率: 66.67％ Saving to /Users/hina/Library/CloudStorage/OneDrive-KyotoUniversity/Documents/KU/commutative-grid/working_dir_cl50/FCC_HCP/filtration/

In [6]:
temp_dict={'ladder_length': 11,
 'dim': 1,
 'radii': np.array([1. , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. ]),
 'dots': ',x,y,multiplicity,area\n0,2,1,333,D\n1,1,11,397,U\n',
 'lines': ',x0,y0,x1,y1,multiplicity\n0,2,1,1,2,333\n'}

In [7]:
vv=Visualizer1(**temp_dict)

In [8]:
fff=vv.render('object')


['1.00', '1.20', '1.40', '1.60', '1.80', '2.00']


In [9]:
fff

In [10]:
fff.write_html('test.html', include_plotlyjs='cdn')

In [11]:
def create_clf_filtration(crystal, homology_dim=1, method="cech", sc_dim_ceil=3, radius_max=2.5):
    upper_layer_pc=crystal.xyz
    lower_df = crystal.thinning(mode="quartet",number_removal=4,inplace=False)
    if lower_df is None:
        return None
    lower_layer_pc = (lower_df[["x", "y", "z"]]).values
    #
    persistence_upper = SimplexTree()
    persistence_lower = SimplexTree()
    persistence_upper.from_point_cloud(upper_layer_pc,method=method,sc_dim_ceil=sc_dim_ceil,radius_max=radius_max)
    persistence_lower.from_point_cloud(lower_layer_pc,method=method,sc_dim_ceil=sc_dim_ceil,radius_max=radius_max)
    critical_radii_all = join_and_unique(persistence_upper.critical_radii(dimension=2),persistence_lower.critical_radii(dimension=homology_dim))
    # prepend with 0.0 and 1.0
    critical_radii_all = np.insert(critical_radii_all,0,1.0)
    critical_radii_all = np.insert(critical_radii_all,0,0.0)
    # append with 2.5
    critical_radii_all = np.append(critical_radii_all,2.5)
    print(critical_radii_all)
    indices_removal = [_ for _ in crystal.df.index if _ not in lower_df.index]
    critical_radii_all = np.linspace(1.0,2.5,51)
    print(indices_removal)
    clf = pointCloud2Filtration(upper_layer_pc,indices_removal,radii=critical_radii_all,max_simplex_dim=sc_dim_ceil)
    return clf


In [12]:
# from cpes import Points3D
# Points3D(crystal.df.iloc[[61, 87, 91, 92]]).plot()


In [13]:
crystal=FCC(7)
homology_dim=1
sc_dim_ceil=homology_dim+1
while True:
    clf=create_clf_filtration(crystal,homology_dim=homology_dim,sc_dim_ceil=sc_dim_ceil)
    if clf is not None:
        break
inv=CLInvariants(clf)
inv.cPD_computation(homology_dim=homology_dim)
V_fcc=Visualizer1(**inv.connected_persistence_diagrams[0].plot_data)
V_fcc.render(export_mode="full_html")

Generating close packing with atom radius 1.
Mode: quartet, changing the layer of 4 atoms.
[0.         1.         1.         1.15470054 1.41421354 1.73205078
 2.         2.5       ]
[81, 82, 88, 130]
Progress: 1.96%
Progress: 3.92%
Progress: 5.88%
Progress: 7.84%
Progress: 9.80%
Progress: 11.76%
Progress: 13.73%
Progress: 15.69%
Progress: 17.65%
Progress: 19.61%
Progress: 21.57%
Progress: 23.53%
Progress: 25.49%
Progress: 27.45%
Progress: 29.41%
Progress: 31.37%
Progress: 33.33%
Progress: 35.29%
Progress: 37.25%
Progress: 39.22%
Progress: 41.18%
Progress: 43.14%
Progress: 45.10%
Progress: 47.06%
Progress: 49.02%
Progress: 50.98%
Progress: 52.94%
Progress: 54.90%
Progress: 56.86%
Progress: 58.82%
Progress: 60.78%
Progress: 62.75%
Progress: 64.71%
Progress: 66.67%
Progress: 68.63%
Progress: 70.59%
Progress: 72.55%
Progress: 74.51%
Progress: 76.47%
Progress: 78.43%
Progress: 80.39%
Progress: 82.35%
Progress: 84.31%
Progress: 86.27%
Progress: 88.24%
Progress: 90.20%
Progress: 92.16%
Progre

Progress: 96.08%


In [ ]:
crystal=HCP(7)
homology_dim=1
sc_dim_ceil=homology_dim+1
while True:
    clf=create_clf_filtration(crystal,homology_dim=homology_dim,sc_dim_ceil=sc_dim_ceil)
    if clf is not None:
        break
inv=CLInvariants(clf)
inv.cPD_computation(homology_dim=homology_dim)
V_fcc=Visualizer1(**inv.connected_persistence_diagrams[0].plot_data)
V_fcc.render(export_mode="full_html")

Generating close packing with atom radius 1.
Mode: quartet, changing the layer of 4 atoms.
> /mnt/ebs/cpes/cpes/close_packings_via_layers.py(369)thinning()
    367         import pdb
    368         pdb.set_trace()
--> 369         df.loc[list(atoms_removed),"layer_joined"]=to_layer
    370         output_df=df.loc[df["layer_joined"]==from_layer]
    371         if inplace:



In [ ]:
# %load_ext snakeviz
# %snakeviz inv_fcc_4.total_decomposition_computation(dim=homology_dim,prime=2)

In [ ]:
inv_fcc_4.total_decomposition_computation(dim=homology_dim,prime=2)
inv_hcp_4.total_decomposition_computation(dim=homology_dim,prime=2)


NameError: name 'inv_fcc_4' is not defined

In [ ]:
inv_fcc_4.decompositions_all[0].nonzero_component

In [ ]:
u2=clf1.get_filtration_by_layer('l')

In [ ]:
from commutazzio.persistence import PD_Points3D
fccp=FCC(num=s_len,perturbation=True)
fccpd=PD_Points3D(fccp.data)

In [ ]:
fcc.data
fvs=[1,1.4,1.725,2.225]
fcc_filt = SimplexTree()
fcc_filt.from_point_cloud(fcc.data,sc_dim_ceil=sc_dim_ceil,radius_max=radius_max)
filt=fcc_filt.to_ordinal_number_indexing(fvs)
filt=filt.to_custom_filtration_values(fvs)

SimplexTree0x122d0e340: 4 filtration values given, smaller than the original A_n quiver length 5.


98

In [ ]:
num_deleted_pts=98
lower_df, _ = thinning_aux(fcc.df, number_removal=num_deleted_pts, style='survived', is_removable='is_interior')
fcc_filt_lower = SimplexTree()
fcc_filt_lower.from_point_cloud((lower_df[["x","y","z"]]).values,sc_dim_ceil=sc_dim_ceil,radius_max=radius_max)
filt_lower=fcc_filt_lower.to_ordinal_number_indexing(fvs)
filt_lower=filt_lower.to_custom_filtration_values(fvs)

SimplexTree0x2a9ce4590: 4 filtration values given, smaller than the original A_n quiver length 5.


In [ ]:
#TODO: try cl4 based on selected critical radii
from gtda.plotting import plot_diagram
opph=filt_lower
diagram=np.array([(birth,death,dim) for dim,(birth,death) in opph.persistence()])
plot_diagram(diagram)

In [ ]:
clf=clf1
invn=CLInvariants(clf)
invn.cPD_computation(dim=homology_dim)
cPD=invn.connected_persistence_diagrams[0]


In [ ]:
# Ways of removal:
# 1. Random constant point removal
# 2. random increment point removal
# 2. random chain removal
# 3. random triangle removal
# 4. random tetrahedron removal
